In [1]:
import pandas as pd
import nltk
import re
import string

In [2]:
fake_data=pd.read_csv('f_train.csv')
fake_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
fake_data.shape

(20800, 5)

In [4]:
final_data=fake_data[0:1000]

In [5]:
final_data.shape

(1000, 5)

In [6]:
final_data.isnull().sum()

id          0
title      27
author    103
text        2
label       0
dtype: int64

In [7]:
final_data=final_data.dropna()

In [8]:
final_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
final_data.reset_index(inplace=True)

In [10]:
final_data.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
#Data Cleaning and preprocessing

In [12]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [13]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [14]:
s_stemmer=SnowballStemmer(language='english')

In [15]:
#from nltk.tag.brill import Word

In [16]:
body=[]
for i in range(0,len(final_data)):
    review=re.sub('[^a-zA-Z]',' ',final_data['text'][i])
    review=review.lower()
    review=review.split()
    review=[s_stemmer.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    body.append(review)

In [17]:
len(body)

870

In [18]:
body[0]

'hous dem aid even see comey letter jason chaffetz tweet darrel lucus octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel case know anthoni weiner sext teenag appar littl thing fact matter c

In [20]:
#tf-idf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
tfidf=TfidfVectorizer()
x=tfidf.fit_transform(body).toarray()
y=final_data['label'].values

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=10)

In [25]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(x_train,y_train)

RandomForestClassifier()

In [26]:
y_pred=rfc.predict(x_test)

In [27]:
from sklearn import metrics

In [28]:
print('test accuracy =',rfc.score(x_test,y_test))
print('training accuracy =',rfc.score(x_train,y_train))
print('accuracy score =',metrics.accuracy_score(y_test,y_pred))

test accuracy = 0.8160919540229885
training accuracy = 1.0
accuracy score = 0.8160919540229885


In [31]:
table=pd.DataFrame(metrics.confusion_matrix(y_test,y_pred),index=['real news','fake news'],columns=['real news','fake news'])
table

,real news,fake news
real news,143,8
fake news,40,70


In [32]:
from sklearn.linear_model import LogisticRegression
lrc=LogisticRegression()
lrc.fit(x_train,y_train)

LogisticRegression()

In [42]:
y_pred_lrc=lrc.predict(x_test)

In [43]:
print('test accuracy =',rfc.score(x_test,y_test))
print('training accuracy =',rfc.score(x_train,y_train))
print('accuracy score =',metrics.accuracy_score(y_test,y_pred_lrc))

test accuracy = 0.8160919540229885
training accuracy = 1.0
accuracy score = 0.9118773946360154


In [44]:
table=pd.DataFrame(metrics.confusion_matrix(y_test,y_pred_lrc),index=['real news','fake news'],columns=['real news','fake news'])
table

,real news,fake news
real news,141,10
fake news,13,97


In [37]:
from sklearn.svm import SVC

In [47]:
svc=SVC(kernel='rbf')
svc.fit(x_train,y_train)

SVC()

In [48]:
y_pred_svc=svc.predict(x_test)

In [45]:
print('test accuracy =',rfc.score(x_test,y_test))
print('training accuracy =',rfc.score(x_train,y_train))
print('accuracy score =',metrics.accuracy_score(y_test,y_pred_svc))

test accuracy = 0.8160919540229885
training accuracy = 1.0
accuracy score = 0.9157088122605364


In [46]:
table=pd.DataFrame(metrics.confusion_matrix(y_test,y_pred_svc),index=['real news','fake news'],columns=['real news','fake news'])
table

,real news,fake news
real news,145,6
fake news,16,94


In [49]:
from sklearn.neighbors import KNeighborsClassifier

In [51]:
knc=KNeighborsClassifier()
knc.fit(x_train,y_train)

KNeighborsClassifier()

In [52]:
y_pred_knc=knc.predict(x_test)

In [54]:
print('test accuracy =',rfc.score(x_test,y_test))
print('training accuracy =',rfc.score(x_train,y_train))model
print('accuracy score =',metrics.accuracy_score(y_test,y_pred_knc))

test accuracy = 0.8160919540229885
training accuracy = 1.0
accuracy score = 0.421455938697318


In [55]:
table=pd.DataFrame(metrics.confusion_matrix(y_test,y_pred_knc),index=['real news','fake news'],columns=['real news','fake news'])
table

,real news,fake news
real news,0,151
fake news,0,110


In [56]:
from sklearn.naive_bayes import MultinomialNB

In [58]:
mnbc=MultinomialNB()
mnbc.fit(x_train,y_train)


MultinomialNB()

In [59]:
y_pred_mnbc=mnbc.predict(x_test)

In [60]:
print('test accuracy =',rfc.score(x_test,y_test))
print('training accuracy =',rfc.score(x_train,y_train))
print('accuracy score =',metrics.accuracy_score(y_test,y_pred_mnbc))

test accuracy = 0.8160919540229885
training accuracy = 1.0
accuracy score = 0.6628352490421456


In [61]:
table=pd.DataFrame(metrics.confusion_matrix(y_test,y_pred_mnbc),index=['real news','fake news'],columns=['real news','fake news'])
table

,real news,fake news
real news,151,0
fake news,88,22


In [64]:
model_accuracies=pd.DataFrame({'RandomForestClassifier':pd.Series(metrics.accuracy_score(y_test,y_pred)),
                    'logisticregression':pd.Series(metrics.accuracy_score(y_test,y_pred_lrc)),
                  'svc':pd.Series(metrics.accuracy_score(y_test,y_pred_svc)),
                  'KNeighbours':pd.Series(metrics.accuracy_score(y_test,y_pred_knc)),
                  'Multinomialnb':pd.Series(metrics.accuracy_score(y_test,y_pred_mnbc))})

In [65]:
model_accuracies

,RandomForestClassifier,logisticregression,svc,KNeighbours,Multinomialnb
0,0.911877,0.911877,0.915709,0.421456,0.662835
